# Compare Gen methods

In [1]:
from plm_model import *
import numpy as np
import torch
import os
import sys
#current_dir = os.path.dirname(__file__)
current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from model import AttentionModel
from attention import trainer
from dcascore import *
# back to original path (in PLM)
sys.path.pop(0)  # Removes the parent_dir from sys.path
from plm_methods import read_tensor_from_txt, set_seed, letters_to_nums, modify_seq

{0: 'A', 20: '-', 1: 'C', 2: 'D', 3: 'E', 4: 'F', 5: 'G', 6: 'H', 7: 'I', 8: 'K', 9: 'L', 10: 'M', 11: 'N', 12: 'P', 13: 'Q', 14: 'R', 15: 'S', 16: 'T', 17: 'V', 18: 'W', 19: 'Y'}


## Load Q, K, V; compute J

In [ ]:
##############################################################
"""
    Load Q, K, V matrices from jdoms (after training)
"""
set_seed()
H = 64
d= 10
n_epochs = 500
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
#cwd = '/Users/marzioformica/Desktop/EPFL/Master/MA2/Labo/my_project/PLM-gen-DCA/Attention-DCA-main/CODE/AttentionDCA_python/src'
cwd='C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\AttentionDCA_python\src'

# Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
# K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
# V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
Q_1 = read_tensor_from_txt( cwd +r'\results\{H}_{d}_{family}_{losstype}_{n_epochs}_youss\Q_tensor.txt'.format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +r'\results\{H}_{d}_{family}_{losstype}_{n_epochs}_youss\K_tensor.txt'.format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +r'\results\{H}_{d}_{family}_{losstype}_{n_epochs}_youss\V_tensor.txt'.format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]

##############################################################
"""
    Initialize the model and compute couplings J from Q, K, V
""" 
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)

torch.Size([64, 63, 63])
21
63


In [42]:
print(Jtens[:,:,30,31])

tensor([[ 8.4190e-02, -1.1062e-03, -7.9344e-02, -6.0122e-03,  1.8884e-02,
          3.7376e-02, -6.7353e-03, -2.6382e-02,  1.9143e-02,  9.7130e-02,
         -4.3858e-02, -3.0229e-02,  4.0515e-02, -1.2474e-02,  5.2964e-02,
         -6.5531e-02,  6.6707e-03,  2.4711e-02, -1.1974e-02, -1.7580e-02,
          2.6321e-02],
        [-1.9488e-02, -2.1032e-04, -2.5435e-02,  5.5132e-03, -1.3635e-02,
         -8.1761e-03,  3.0495e-03, -1.3606e-02, -1.4065e-02, -4.8213e-02,
         -4.1118e-03, -8.1658e-03,  4.1129e-03,  1.3696e-03, -1.2804e-02,
         -2.0105e-02, -7.4580e-03, -1.5246e-02, -4.5735e-03,  1.1006e-02,
         -2.0161e-03],
        [-3.4951e-02,  5.6149e-03, -9.6376e-02, -8.7806e-02,  4.6556e-02,
         -1.1157e-02, -9.2960e-03, -7.7389e-04,  1.4078e-01, -4.5730e-02,
         -2.6454e-03,  1.3194e-02, -1.9140e-02,  2.7268e-02,  1.5225e-01,
          5.0871e-02, -1.6868e-02, -1.0864e-02, -2.9854e-03, -8.3227e-02,
          1.3813e-04],
        [-2.4088e-02,  1.1599e-02, -3.6280e

In [5]:
import random
def read_tensor_from_txt(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Read the dimensions from the first line
    dims = list(map(int, lines[0].strip().split()))
    
    tensor_data = []
    current_slice = []
    for line in lines[1:]:
        line = line.strip()
        if line.startswith("Slice"):
            if current_slice:
                tensor_data.append(current_slice)
                current_slice = []
        elif line:
            current_slice.append(list(map(float, line.split(','))))
    if current_slice:
        tensor_data.append(current_slice)

    tensor = torch.tensor(tensor_data).view(*dims)
    return tensor
def set_seed(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
def modify_seq(seq,ratio,aa_list=np.arange(21)):
    L=len(seq)
    seq_func=seq.copy()
    nb_change=int(L*ratio)
    ind_change=np.random.choice(np.arange(L),nb_change)
    for i in range(nb_change):
        seq_func[ind_change[i]]=np.random.choice(aa_list)
    return seq_func
def invert_dict(d):
    inverted = {}
    for key, value in d.items():
        if value in inverted:
            inverted[value].append(key)
        else:
            inverted[value] = [key]
    return inverted
def numbers_to_letters(numbers, inverted_dict):
    result = ''
    for number in numbers:
        if number in inverted_dict:
            letters = inverted_dict[number]
            result += random.choice(letters)
        else:
            raise ValueError(f"Number {number} not found in the inverted dictionary.")
    return result
def seq_num_to_letters(file_path,dictionary):
    inv_dict=invert_dict(dictionary)
    seq_array=np.loadtxt(file_path).astype(np.int64)
    letter_seq_array=[]
    for seq in seq_array:
        letter_seq_array.append(numbers_to_letters(seq,inv_dict))
    output_path=file_path.replace(".txt",'')+"output.txt"
    np.savetxt(output_path, np.array(letter_seq_array), fmt='%s')

In [29]:
letter_to_num_ale = {
    'A': 1,  'B': 21, 'C': 2,  'D': 3,  'E': 4,
    'F': 5,  'G': 6,  'H': 7,  'I': 8,  'J': 21,
    'K': 9,  'L': 10, 'M': 11, 'N': 12, 'O': 21,
    'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17,
    'U': 21, 'V': 18, 'W': 19, 'X': 21, 'Y': 20,
    '-': 21  # Gap symbol
}
letter_to_num = {
    'A': 0,  'B': 20, 'C': 1,  'D': 2,  'E': 3,
    'F': 4,  'G': 5,  'H': 6,  'I': 7,  'J': 20,
    'K': 8,  'L': 9, 'M': 10, 'N': 11, 'O': 20,
    'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16,
    'U': 20, 'V': 17, 'W': 18, 'X': 20, 'Y': 19,
    '-': 20  # Gap symbol
}

letter_to_num_conversion = {
    'A': 0,  'C': 1,  'D': 2,  'E': 3,
    'F': 4,  'G': 5,  'H': 6,  'I': 7,  
    'K': 8,  'L': 9, 'M': 10, 'N': 11, 
    'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16,
    'V': 17, 'W': 18,  'Y': 19,
    '-': 20  # Gap symbol
}

#switched the indexes to minus 1
def prob_cond_unnormalized(aa,prev_aa,J_ten,beta=1):
    i=len(prev_aa)
    sum=0
    for j in range(i):
        sum+=J_ten[aa,prev_aa[j],i,j]
    return np.exp(beta*sum)
def Z_i_prob(aa_pot,prev_aa,J_tens):
    list_prob=[prob_cond_unnormalized(aa,prev_aa,J_tens) for aa in aa_pot ]
    return list_prob
def sample(aa_pot,Z_list):
    Z=np.sum(Z_list)
    Z_list=Z_list/Z
    return np.random.choice(aa_pot,p=Z_list)



In [14]:
letter_to_num_ale = {
    'A': 1,  'B': 21, 'C': 2,  'D': 3,  'E': 4,
    'F': 5,  'G': 6,  'H': 7,  'I': 8,  'J': 21,
    'K': 9,  'L': 10, 'M': 11, 'N': 12, 'O': 21,
    'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17,
    'U': 21, 'V': 18, 'W': 19, 'X': 21, 'Y': 20,
    '-': 21  # Gap symbol
}
letter_to_num = {
    'A': 0,  'B': 20, 'C': 1,  'D': 2,  'E': 3,
    'F': 4,  'G': 5,  'H': 6,  'I': 7,  'J': 20,
    'K': 8,  'L': 9, 'M': 10, 'N': 11, 'O': 20,
    'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16,
    'U': 20, 'V': 17, 'W': 18, 'X': 20, 'Y': 19,
    '-': 20  # Gap symbol
}
# switched here as well for the computation of the prob
def initial_sample(L):
    list_nb=np.arange(22)
    init_sample=np.random.choice(list_nb,L)
    return init_sample

def plm_seq(seq,J):
    sum=0
    L=J.shape[-1]
    for i in range(L):
        for j in range(L):
            sum+=J[seq[i],seq[j],i,j]
    return sum

def plm_aa_calc_diff(aa_new,ind_change,seq,J_tens):
    L=J_tens.shape[-1]
    sum=0
    for j in range(L):
        delta_J1=J_tens[aa_new,seq[j],ind_change,j]-J_tens[seq[ind_change],seq[j],ind_change,j]
        delta_J2=J_tens[seq[j],aa_new,j,ind_change]-J_tens[seq[j],seq[ind_change],j,ind_change]
        sum+=delta_J1+delta_J2
    return sum

def plm_aa_calc_diff_alter(aa_new,ind_change,seq,J_tens):
    L=J_tens.shape[-1]
    sum=0
    for j in range(L):
        delta_J1=J_tens[aa_new,seq[j],ind_change,j]
        delta_J2=J_tens[seq[j],aa_new,j,ind_change]
        sum+=delta_J1+delta_J2
    return sum


def plm_ind_change(aa_pot,ind,seq,J,beta=1,old_plm=None,test=False):#probablement false
    if old_plm==None:
        old_plm=plm_seq(seq,J)
    list_aa_plm=np.array([])
    list_aa_plm_alter=np.array([])
    
    for i in range(len(aa_pot)):
        list_aa_plm=np.append(list_aa_plm,plm_aa_calc_diff(aa_pot[i],ind,seq,J))
        list_aa_plm_alter=np.append(list_aa_plm_alter,plm_aa_calc_diff_alter(aa_pot[i],ind,seq,J))
        # print("diff for two sequences",plm_aa_calc_diff(aa_pot[i],ind,seq,J))
        
        if test==True:
            seq_t=seq.copy()
            seq_t[ind]=aa_pot[i]
            print("diff two methods:",old_plm+plm_aa_calc_diff(aa_pot[i],ind,seq,J)-plm_seq(seq_t,J))
    prob_unn=np.exp(beta*list_aa_plm)    #à checker signe dans l'exponentiel
    pro_alter=np.exp(beta*list_aa_plm_alter)
    def assert_no_nan(arr, name="array"):
        """
        Checks if a NumPy array contains any NaNs.
        If it does, raises a ValueError with debug info.
        
        Parameters:
            arr (np.ndarray): Array to check.
            name (str): Optional name to include in the error message.
        """
        if np.isnan(arr).any():
            nan_indices = np.argwhere(np.isnan(arr))
            sample = arr.flatten()[np.isnan(arr.flatten())][:5]  # First 5 NaNs
            raise ValueError(
                f"NaN detected in {name}!\n"
                f"First few NaN values: {sample}\n"
                f"Indices of NaNs: {nan_indices[:5]}\n"
                f"Total NaNs: {np.isnan(arr).sum()}\n"
                f"old plm: {old_plm}\n"
                f"prob_list: {prob_unn}"
            )
    assert_no_nan(prob_unn/np.sum(prob_unn))
    print("difference of proba(should be around zero):",prob_unn/np.sum(prob_unn)-pro_alter/np.sum(pro_alter))
    return prob_unn/np.sum(prob_unn), list_aa_plm

def plm_ind_change_quick(aa_pot,ind,seq,J,beta=1):
    list_aa_plm=np.zeros(len(aa_pot))
    #list_aa_plm=np.array([])
    for i in range(len(aa_pot)):
        #list_aa_plm=np.append(list_aa_plm,plm_aa_calc_diff_alter(aa_pot[i],ind,seq,J))
        list_aa_plm[i]=plm_aa_calc_diff_alter(aa_pot[i],ind,seq,J)
    prob_unn=np.exp(beta*list_aa_plm) 
    return prob_unn/np.sum(prob_unn)

def plm_sample(aa_pot,proba_list):
    ind_choice=np.random.choice(range(len(aa_pot)),p=proba_list)
    return aa_pot[ind_choice],ind_choice


def generate_plm(J_tens,maxiter=10000,beta=1,initial_seq=None,quick=True):
    L=J_tens.shape[-1]
    if initial_seq is None:
        sequence=initial_sample(L)
    else:
        sequence=initial_seq
    list_nb=np.arange(21)
    seq_list=[]
    seq_list.append(sequence.copy())
    position_list=np.arange(L)
    ind_change_list=np.random.choice(position_list,size=maxiter)
    if quick:
        for i in tqdm(range(maxiter)):
            ind_change=ind_change_list[i]
            prob_list=plm_ind_change_quick(list_nb,ind_change,sequence,J_tens,beta=beta)
            new_aa,ind_chosen=plm_sample(list_nb,prob_list)
            sequence[ind_change]=new_aa
            seq_list.append(sequence.copy())
        return np.array(seq_list)
    else:
        old_plm=plm_seq(sequence,J_tens)
        plm_list=[]
        plm_list.append(old_plm)
        for i in tqdm(range(maxiter)):
            old_sequence=sequence.copy()
            ind_change=np.random.choice(np.arange(L))
            prob_list,pot_plm=plm_ind_change(list_nb,ind_change,sequence,J_tens,beta=beta,old_plm=old_plm)
            new_aa,ind_chosen=plm_sample(list_nb,prob_list)
            sequence[ind_change]=new_aa
            seq_list.append(sequence.copy())
            old_plm=pot_plm[ind_chosen]
            #print("difference in plm calc method:",(old_plm-plm_seq(sequence,J_tens))/old_plm) #seems close enough but not in the computer numerical errors range 
            plm_list.append(pot_plm[ind_chosen])
        return np.array(seq_list),plm_list

In [31]:
def nums_to_letters(sequence):
    """Receives a sequence of integers (e.g., [1, 2, 4]) and returns a string of corresponding amino acid letters."""
    num_to_letter = {v: k for k, v in letter_to_num.items()}
    return ''.join([num_to_letter.get(num, 'X') for num in sequence])

init_seq = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = letters_to_nums(init_seq)
ratio = 0.1
init_sequence_num = modify_seq(init_sequence_num, ratio)
print(init_sequence_num)
print(nums_to_letters(init_sequence_num))
inv_letter_to_num_conversion = invert_dict(letter_to_num_conversion)
print(numbers_to_letters(init_sequence_num, inv_letter_to_num_conversion))

[ 2 19 19  1 17  9  5 17 10  8  2  0  2  0  8 15  7  8  8  0  4 14  8  9
  0 14 15 19  6  6  2 17 11 12  5  2  8  3  0  3 14  8  4  8  3  1 11  3
  0 11  3 17  9 15  2 12  3 20 14  8  8 19  2]
DYYCVLGVMKDADAKSIKKAFRKLARSYHHDVNPGDKEAERKFKECNEANEVLSDPE-RKKYD
DYYCVLGVMKDADAKSIKKAFRKLARSYHHDVNPGDKEAERKFKECNEANEVLSDPE-RKKYD


In [ ]:
gen_sequences_y = generate_plm(Jtens,maxiter=1000,beta = 1, initial_seq=init_sequence_num.copy() ,quick=True)

100%|██████████| 1000/1000 [00:28<00:00, 35.54it/s]


TypeError: non-string returned while reading data

In [37]:
folder_name = 'results_plm'
os.makedirs(folder_name, exist_ok=True)
file_path = os.path.join(folder_name, 'generated_sequences.txt')
np.savetxt(file_path, gen_sequences_y,fmt="%.5f")
seq_num_to_letters(file_path,letter_to_num_conversion)

In [28]:
for i in range(0, len(gen_sequences_y), 100):
    # Convert the sequence at index i to letters and print
    sequence_in_letters = nums_to_letters(gen_sequences_y[i])
    print(f"Sequence {i + 1} (in letters): {sequence_in_letters}")

Sequence 1 (in letters): DYYQVLGVKKDHDAKSIKKAFRKLARKYHPDVNPGDNEAERKFKEAREANEVLSDKEKRKKYD
Sequence 101 (in letters): ARYDVKALAVDHDQPSLLEALHELALKLRDDYRAGDRFATAVYREAQADTLKLQEYHAHKHRD
Sequence 201 (in letters): ARYALLAEAVDHDLLSLLAAKHVLALQLRDAYRADYRDAAAVDRELQADTLRLQDYHHLIFRA
Sequence 301 (in letters): ARAELRAAAVDDLLASLLAAGRAKALTLRDAYKADLRDEAFRQRERRAATLELRDPAAQIFRA
Sequence 401 (in letters): ARDELRADAVADLRAAVAAALRAPALYLADAATAALRPAAFRDRARLAAALLLRDPAAQRARA
Sequence 501 (in letters): ARDELRADAAADAQAAVAAAGRADLAALADAAAAAVRPAAARDRARLAAALLARDPAAERARA
Sequence 601 (in letters): ARATVAADLAAAAAAAVAAALRDDAAALADAAAAAVRPAAARDRARLAAAAAARAPAAERARA
Sequence 701 (in letters): ARLTLAADAAAAAAAAAAALGRLTAAALADAAARALRRAAARDAARAAAAAAAAAPTAARARA
Sequence 801 (in letters): ARLALAADAAAAAAAAAQALLAATAAARADDAAVALRFAAARDAARRAAAAAAAAPAAARARA
Sequence 901 (in letters): ARTALAADLAAAAAAAAAALTAVTAAAAADAAAVALRRAAARDAARRAAAAAAAAPAAAEARA
Sequence 1001 (in letters): ARTALAADLAAAAAAAAPAAPAATAAAAADAAAVALRLAAAADAARRAAAAVARAPAAAAARA
